# TEXT CLASSIFICATION USING PYSPARK AND MLLIB

The goal of Lab 3 is the classification of ARTICLES downloaded from NYT into a fixed number of predefined categories. In this notebook, we walk through an example of text classifcation using Spark machine learning algorithms. We will  classify the documents into 4 categories -Multi Class classification.
Steps:

## Build a data pipeline using Downloaded Data from  NY Times articles using the APIs provided by the data sources.  
#####       a. Create PYSAPRK DataFrames from the downloaded data 
#####      b. Split the Learning Dataset into training and test data 
#####      c. Extract features that will determine the class or category of the article {politics, sports,
#####      business, arts}
## Build a model for classification using Logistic Regression, Naive Bayes and Random Foresy classification algorithms on Learning Dataset.
## Assess the accuracy of MODEL using the UNKNOWN data 
## Compare the classification accuracy of at least two well-known classification algorithms,  for a given text data set.

### Create DATA FRAME in PYSPARK

In [68]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [69]:
path = os.chdir('/Users/vikrant/data/')

from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [70]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, StopWordsRemover, IDF, Tokenizer
from pyspark.ml.tuning import ParamGridBuilder,CrossValidator
from pyspark.mllib.linalg import Vector

### Read Downloaded Data Files ; Create Learning DataSet

In [71]:
dirpath = '/Users/vikrant/data/NYT/*'
NYTRawData = sc.wholeTextFiles(dirpath)

In [72]:
#print("The number of documents read in is " + NYTRawData.count() + ".")
NYTRawData.count()

589

### Create UNKNOWN Dataset

In [73]:
dirpathUN = '/Users/vikrant/data/UnknownData/*'
NYTRawDataUNKNOWN = sc.wholeTextFiles(dirpathUN)

In [74]:
#print("The number of documents read in is " + NYTRawData.count() + ".")
NYTRawDataUNKNOWN.count()

49

### Display Sample Data from Learning Dataset 

In [75]:
NYTRawData.takeSample(False,1, seed = 231279)

[('file:/Users/vikrant/data/NYT/business/5a96061810f40f00018c366b',
  'In N.R.A. Fight, Delta Finds There Is No Neutral Ground\nATLANTA — Delta Air Lines, one of Georgia’s largest private-sector employers, is among the glossiest corporate jewels in the state, contributing billions of dollars a year to the economy. Its hundreds of daily flights in and out of Atlanta’s airport are the main reason Georgia’s capital can brag, with justification, that it is a truly international city.\n“Delta is the Atlanta international airport, and our airport is, you know, the fuel that generates our commercial community,” said Sam Massell, the city’s former mayor.\nBut prestige, boosterism and corporate coddling — all cherished concepts in Georgia — took a back seat this week to the national debate over gun control. In the wake of the Florida school shooting, the airline announced it was ending a promotional discount with the National Rifle Association, and suddenly found itself in the rare position of 

### Display Sample Data from UNKNOWN Dataset 

In [76]:
NYTRawDataUNKNOWN.takeSample(False,1, seed = 231279)

[('file:/Users/vikrant/data/UnknownData/business/5ae88376068401528a2aba41',
  'Legendary U.S. Guitar-Maker Gibson Files for Bankruptcy\n(Reuters) - Gibson Brands Inc, the maker of guitars played by the likes of B.B. King and Elvis Presley, filed for Chapter 11 bankruptcy protection on Tuesday with a plan to reorganize its musical instrument business under the new ownership of its lenders.\nNashville-based Gibson, whose legendary brands include Les Paul and SG, has been suffering under $500 million in debt linked to the acquisition of its consumer electronics business overseas, where sales have been in sharp decline.\nIn a filing in U.S. Bankruptcy Court in Delaware, Gibson said the overseas consumer electronics business will be wound down, allowing it to re-focus on its core guitar-making and audio businesses.\nThe audio business includes KRK, Cerwin Vega and Stanton headphones, loud speakers and turntables for amateur and professional musicians and sound engineers. \n"This process wil

In [77]:
filepath = NYTRawData.map(lambda x:x[0]).collect()

#### Filter RDD to Capture Text

In [78]:
text = NYTRawData.map(lambda x:x[1]).collect()

In [79]:
textUN = NYTRawDataUNKNOWN.map(lambda x:x[1]).collect()

#### Convert to DataFrame
##### Learning Dataframe = "df"
##### Unknown Dataframe =" dfUN"

In [80]:
from pyspark.sql.types import Row

#here you are going to create a function
def f(x):
    d = {}
    for i in range(len(x)):
        d[str(i)] = x[i]
    return d

#Now populate that
df = NYTRawData.map(lambda x: Row(**f(x))).toDF()#.withColumn("Label",lit("Politics"))
dfUN = NYTRawDataUNKNOWN.map(lambda x: Row(**f(x))).toDF()

In [81]:
df.columns

['0', '1']

In [82]:
dfUN.columns

['0', '1']

#### Prepare Learning Dataset for Modeling using Classification Models
###### Split Columns to get Category of each Article

In [83]:
from pyspark.sql.functions import split
split_col = split(df['0'], '/')

In [84]:
df = df.withColumn('NAME6', split_col.getItem(5))
df = df.withColumn('NAME7', split_col.getItem(6))

In [85]:
df.show()

+--------------------+--------------------+-----+--------------------+
|                   0|                   1|NAME6|               NAME7|
+--------------------+--------------------+-----+--------------------+
|file:/Users/vikra...|R. Kelly Faces a ...| arts|5ae88aee068401528...|
|file:/Users/vikra...|Pop, Rock and Jaz...| arts|5a4f82147c459f29e...|
|file:/Users/vikra...|‘Così’ at the Met...| arts|5aac272d068401528...|
|file:/Users/vikra...|‘Breaking Ice’ Ma...| arts|5a4f5ac57c459f29e...|
|file:/Users/vikra...|8 Classical Music...| arts|5acfc8d8068401528...|
|file:/Users/vikra...|8 Classical Music...| arts|5aaaded9068401528...|
|file:/Users/vikra...|SZA Almost Quit M...| arts|5a6798f610f40f000...|
|file:/Users/vikra...|Luigi Nono’s Hars...| arts|5a905b9a10f40f000...|
|file:/Users/vikra...|14 Pop, Rock and ...| arts|5acfc8da068401528...|
|file:/Users/vikra...|Yannick, Sooner: ...| arts|5a87286d10f40f000...|
|file:/Users/vikra...|Seeking Orchestra...| arts|5ad74f68068401528...|
|file:

In [86]:
df.printSchema()

root
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- NAME6: string (nullable = true)
 |-- NAME7: string (nullable = true)



In [87]:
drop_list = ['0', 'NAME7']#, 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
df = df.select([column for column in df.columns if column not in drop_list])
df.show(5)
df.printSchema()

+--------------------+-----+
|                   1|NAME6|
+--------------------+-----+
|R. Kelly Faces a ...| arts|
|Pop, Rock and Jaz...| arts|
|‘Così’ at the Met...| arts|
|‘Breaking Ice’ Ma...| arts|
|8 Classical Music...| arts|
+--------------------+-----+
only showing top 5 rows

root
 |-- 1: string (nullable = true)
 |-- NAME6: string (nullable = true)



In [88]:

from pyspark.sql.functions import col
df = df.select(col("1").alias("Article"), col("NAME6").alias("Category"))
df.show()

+--------------------+--------+
|             Article|Category|
+--------------------+--------+
|R. Kelly Faces a ...|    arts|
|Pop, Rock and Jaz...|    arts|
|‘Così’ at the Met...|    arts|
|‘Breaking Ice’ Ma...|    arts|
|8 Classical Music...|    arts|
|8 Classical Music...|    arts|
|SZA Almost Quit M...|    arts|
|Luigi Nono’s Hars...|    arts|
|14 Pop, Rock and ...|    arts|
|Yannick, Sooner: ...|    arts|
|Seeking Orchestra...|    arts|
|Live Nation Rules...|    arts|
|The Playlist: Nic...|    arts|
|7 Classical Music...|    arts|
|Review: ‘Written ...|    arts|
|Logic, XXXTentaci...|    arts|
|A Long-Lost Compo...|    arts|
|14 Pop, Rock and ...|    arts|
|Lucy Dacus, Socce...|    arts|
|Through Opera, De...|    arts|
+--------------------+--------+
only showing top 20 rows



##### Learning Data Set 

##### (Collection of approx 589 articles in 4 categories from NYT)

In [89]:
from pyspark.sql.functions import col
df.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|Category|count|
+--------+-----+
|politics|  188|
|  sports|  155|
|    arts|  151|
|business|   95|
+--------+-----+



#### Prepare UNKNOWN DataSet for Testing using CLASSIFICATION MODELS

In [90]:

split_col = split(dfUN['0'], '/')
dfUN = dfUN.withColumn('NAME6', split_col.getItem(5))
dfUN = dfUN.withColumn('NAME7', split_col.getItem(6))
dfUN.printSchema()
drop_list = ['0', 'NAME7']#, 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
dfUN = dfUN.select([column for column in dfUN.columns if column not in drop_list])
#dfUN.show(5)
dfUN = dfUN.select(col("1").alias("Article"), col("NAME6").alias("Category"))
#dfUN.show()
from pyspark.sql.functions import col
dfUN.groupBy("Category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

root
 |-- 0: string (nullable = true)
 |-- 1: string (nullable = true)
 |-- NAME6: string (nullable = true)
 |-- NAME7: string (nullable = true)

+--------+-----+
|Category|count|
+--------+-----+
|  sports|   15|
|business|   12|
|politics|   12|
|    arts|   10|
+--------+-----+



### DIvide  Learning Data Set into Training and Test

In [91]:
# set seed for reproducibility
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed = 231279)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Training Dataset Count: 472
Test Dataset Count: 117


In [92]:
trainingData.show()

+--------------------+--------+
|             Article|Category|
+--------------------+--------+
|10 Treasures, Une...|    arts|
|100 Years After D...|    arts|
|13 Pop, Rock and ...|    arts|
|13 Pop, Rock and ...|    arts|
|13 Pop, Rock and ...|    arts|
|14 Pop, Rock and ...|    arts|
|15 Pop, Rock and ...|    arts|
|2018 N.F.L. Draft...|  sports|
|28 Years After Hi...|    arts|
|5 ‘Schoolhouse Ro...|    arts|
|6 Classical Music...|    arts|
|6 Classical Music...|    arts|
|6 Classical Music...|    arts|
|6 Classical Music...|    arts|
|7 Classical Music...|    arts|
|7 Classical Music...|    arts|
|8 Classical Music...|    arts|
|8 Classical Music...|    arts|
|8 Classical Music...|    arts|
|A Battle Over Tra...|business|
+--------------------+--------+
only showing top 20 rows



In [93]:
testData.show()

+--------------------+--------+
|             Article|Category|
+--------------------+--------+
|14 Pop, Rock and ...|    arts|
|14 Pop, Rock and ...|    arts|
|14 Pop, Rock and ...|    arts|
|14 Pop, Rock and ...|    arts|
|6 Classical Music...|    arts|
|7 Classical Music...|    arts|
|7 Classical Music...|    arts|
|A Tone Parallel t...|    arts|
|Air France Cuts B...|business|
|Athletics-AIU Con...|  sports|
|Baffert Talks Der...|  sports|
|Classical Music i...|    arts|
|Classical Music i...|    arts|
|Clayton Homes Mim...|business|
|Cleveland Conquer...|    arts|
|Column: Payne's I...|  sports|
|Column: US League...|  sports|
|ESPN Tries to Get...|  sports|
|Elliott Says Back...|business|
|FIFA Bans Brazili...|  sports|
+--------------------+--------+
only showing top 20 rows



#### DISPLAY UNknown DF

In [94]:
dfUN.show()

+--------------------+--------+
|             Article|Category|
+--------------------+--------+
|Avicii: Overexpos...|    arts|
|‘Bobby Tarantino ...|    arts|
|Listen to This Aw...|    arts|
|Beyoncé and Kendr...|    arts|
|Abba Records New ...|    arts|
|In Defense of Sou...|    arts|
|Hear What Music W...|    arts|
|Berklee College E...|    arts|
|Justin Timberlake...|    arts|
|The Complex Inter...|    arts|
|Russia's NLMK Say...|business|
|U.S. Factory Orde...|business|
|Legendary U.S. Gu...|business|
|New Recycling Ser...|business|
|Permira to Buy Ba...|business|
|China Asks U.S. t...|business|
|Leader of Latino ...|business|
|OC Oerlikon Prepa...|business|
|Kellogg Tops Prof...|business|
|Why BlackRock’s M...|business|
+--------------------+--------+
only showing top 20 rows



## Classification  Using Logistic Regression
**LogisticRegression** is a method used to predict a binary response. The current implementation of logistic regression in spark.ml only supports binary classes. Support for multiclass regression will be added in the future.
### Train the Learning Dataset (TRAIN And TEst)
### Build Pipeline using TF IDF 

In machine learning, it is common to run a sequence of algorithms to process and learn from data. Spark ML represents such a workflow as a Pipeline, which consists of a sequence of PipelineStages (Transformers and Estimators) to be run in a specific order. The pipeline we are using in this example consists of four stages: Tokenizer, StopWordsRemover, HashingTF, Inverse Document Frequency (IDF) and LogisticRegression.

**Tokenizer** splits the raw text documents into words, adding a new column with words into the dataset.

**StopWordsRemover** takes as input a sequence of strings and drops all the stop words from the input sequences. Stop words are words which should be excluded from the input, typically because the words appear frequently and don’t carry as much meaning. A list of stop words by default. Optionally you can provide a list of stopwords. We will just use the defualt list of stopwords.

**HashingTF** takes sets of terms and converts those sets into fixed-length feature vectors. 

**Inverse Document Frequency (IDF)** is a numerical measure of how much information a term provides. If a term appears very often across the corpus, it means it doesn’t carry special information about a particular document. IDF down-weights terms which appear frequently in a corpus.



###### Our model will make predictions and score on the test set; we then look at the top 10 predictions from the highest probability.

In [95]:
from pyspark.ml.feature import RegexTokenizer

In [96]:
regexTokenizer = RegexTokenizer(inputCol="Article", outputCol="words", pattern="\\W")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
add_stopwords = ["http","https","amp","rt","t","c","the"] 
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered").setStopWords(add_stopwords)
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)

In [97]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
label_stringIdx = StringIndexer(inputCol = "Category", outputCol = "label")

### Define Learning Model

In [98]:
pipeline = Pipeline(stages=[regexTokenizer, remover, hashingTF, idf, label_stringIdx, lr])

In [99]:
model = pipeline.fit(trainingData)


### Train the Model using Logistic Regression

In [100]:
lrdata = model.transform(trainingData).select("words","features","label","probability","prediction").show()

+--------------------+--------------------+-----+--------------------+----------+
|               words|            features|label|         probability|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|[10, treasures, u...|(10000,[2,93,126,...|  2.0|[0.01011542778836...|       2.0|
|[100, years, afte...|(10000,[32,87,90,...|  2.0|[0.00472514455689...|       2.0|
|[13, pop, rock, a...|(10000,[26,57,69,...|  2.0|[0.00166947516238...|       2.0|
|[13, pop, rock, a...|(10000,[31,37,62,...|  2.0|[0.00353282966956...|       2.0|
|[13, pop, rock, a...|(10000,[57,62,122...|  2.0|[0.00358468847175...|       2.0|
|[14, pop, rock, a...|(10000,[12,23,60,...|  2.0|[0.00248503592248...|       2.0|
|[15, pop, rock, a...|(10000,[29,76,104...|  2.0|[2.46429226092113...|       2.0|
|[2018, n, f, l, d...|(10000,[33,87,123...|  1.0|[0.00668749713916...|       1.0|
|[28, years, after...|(10000,[65,104,12...|  2.0|[0.02344520328745...|       2.0|
|[5, schoolhouse

### Perform Prediction on TEST Data

In [101]:
predictions = model.transform(testData)

In [102]:
predictions.filter(predictions['prediction'] == 1) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|  sports|(10000,[32,82,126,157,161,1...|[0.40628029138195526,0.5298...|  1.0|       1.0|
|  sports|(10000,[47,293,520,569,585,...|[0.2452264764527254,0.33698...|  1.0|       1.0|
|business|(10000,[26,57,76,103,114,12...|[0.21088471936828349,0.4169...|  3.0|       1.0|
|  sports|(10000,[15,70,161,186,195,2...|[0.19243354388454073,0.6804...|  1.0|       1.0|
|  sports|(10000,[36,130,145,157,533,...|[0.16243186709011614,0.4579...|  1.0|       1.0|
|  sports|(10000,[10,20,34,59,132,147...|[0.12633218377026162,0.8164...|  1.0|       1.0|
|  sports|(10000,[76,77,141,204,262,3...|[0.10803912714356499,0.8309...|  1.0|       1.0|
|  sports|(10000,[157,335,347,353,388...|[0.09473190813279275,0.6520...|  1.0|       1.0|
|  sports|

## Accuracy on Test data using Logistic Regression
#### Keep in mind that the model has not seen the documents in the test data set.

In [103]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9057057246712419

## Accuracy of UNKNOWN Data using Logistic Regression
#### Keep in mind that the model has not seen the documents in the Unknown data set.

In [104]:
predictions = model.transform(dfUN)
predictions.filter(predictions['prediction'] == 1) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|  sports|(10000,[1,15,94,147,356,585...|[0.231449591539279,0.492834...|  1.0|       1.0|
|  sports|(10000,[45,157,288,293,468,...|[0.12872954468858455,0.5716...|  1.0|       1.0|
|  sports|(10000,[59,63,147,150,157,1...|[0.11903603197440375,0.6733...|  1.0|       1.0|
|  sports|(10000,[20,143,157,220,239,...|[0.11720490717896263,0.7137...|  1.0|       1.0|
|  sports|(10000,[7,24,76,99,113,140,...|[0.11697998701478371,0.6641...|  1.0|       1.0|
|  sports|(10000,[36,127,163,186,204,...|[0.09829173344749605,0.7605...|  1.0|       1.0|
|  sports|(10000,[147,208,235,343,474...|[0.09773824375209174,0.7883...|  1.0|       1.0|
|  sports|(10000,[20,120,147,161,177,...|[0.07440919158802979,0.8308...|  1.0|       1.0|
|  sports|

0.918297808424174

## CLASSIFICATION USING "NAIVE BAYES"
### RANDOM SPLIT DATA AGAIN
### DEFINE PIPELINE
### TRAIN MODEL

In [105]:
from pyspark.ml.classification import NaiveBayes
(trainingData2, testData2) = df.randomSplit([0.8, 0.2], seed = 231279)
nb = NaiveBayes(smoothing=1)
pipelinenb = Pipeline(stages=[regexTokenizer, remover, hashingTF, idf, label_stringIdx, nb])

model = pipelinenb.fit(trainingData2)


In [106]:
nbdata = model.transform(trainingData).select("words","features","label","probability","prediction").show()

+--------------------+--------------------+-----+--------------------+----------+
|               words|            features|label|         probability|prediction|
+--------------------+--------------------+-----+--------------------+----------+
|[10, treasures, u...|(10000,[2,93,126,...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[100, years, afte...|(10000,[32,87,90,...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[13, pop, rock, a...|(10000,[26,57,69,...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[13, pop, rock, a...|(10000,[31,37,62,...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[13, pop, rock, a...|(10000,[57,62,122...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[14, pop, rock, a...|(10000,[12,23,60,...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[15, pop, rock, a...|(10000,[29,76,104...|  2.0|   [0.0,0.0,1.0,0.0]|       2.0|
|[2018, n, f, l, d...|(10000,[33,87,123...|  1.0|   [0.0,1.0,0.0,0.0]|       1.0|
|[28, years, after...|(10000,[65,104,12...|  2.0|[4.98052256750894...|       2.0|
|[5, schoolhouse

### Perform Prediction of TEst Data using Model Trained on "Naive Bayes"

In [107]:
predictions = model.transform(testData2)
predictions.filter(predictions['prediction'] == 0) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|  sports|(10000,[87,115,186,312,360,...|[1.0,4.5637026367223514E-20...|  1.0|       0.0|
|    arts|(10000,[15,76,278,312,321,3...|[1.0,4.80191748543393E-31,2...|  2.0|       0.0|
|  sports|(10000,[1,2,157,286,312,332...|[1.0,1.0338429046465282E-34...|  1.0|       0.0|
|politics|(10000,[71,195,200,219,321,...|[1.0,1.1843904863324864E-46...|  0.0|       0.0|
|politics|(10000,[26,52,63,65,76,78,1...|[1.0,4.443174102829313E-126...|  0.0|       0.0|
|politics|(10000,[103,126,141,157,241...|[1.0,4.150040771021918E-174...|  0.0|       0.0|
|politics|(10000,[8,15,21,44,52,68,12...|[1.0,2.96362269377377E-175,...|  0.0|       0.0|
|    arts|(10000,[1,18,20,32,41,47,56...|[1.0,2.582302963410581E-175...|  2.0|       0.0|
|politics|

## Accuracy of Test data Using "Naive Bayes"
#### Keep in mind that the model has not seen the documents in the test data set.

In [108]:
evaluatornb = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluatornb.evaluate(predictions)

0.9322248379320339

## Accuracy of UNKNOWN data Using "Naive Bayes"
#### Keep in mind that the model has not seen the documents in the Unknown data set.

In [109]:
predictions = model.transform(dfUN)
predictions.filter(predictions['prediction'] == 0) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
evaluatornb = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluatornb.evaluate(predictions)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|politics|(10000,[21,103,115,126,130,...|[1.0,1.7200949986592894E-11...|  0.0|       0.0|
|politics|(10000,[78,165,207,216,273,...|[1.0,1.4737382399933485E-13...|  0.0|       0.0|
|politics|(10000,[1,76,141,273,344,37...|[1.0,3.490530220472768E-207...|  0.0|       0.0|
|politics|(10000,[15,17,95,137,157,17...|[1.0,7.759440053264402E-226...|  0.0|       0.0|
|politics|(10000,[1,87,115,126,147,16...|[1.0,1.4165077832079794E-26...|  0.0|       0.0|
|politics|(10000,[1,24,76,78,87,93,10...|[1.0,0.0,0.0,3.177223790333...|  0.0|       0.0|
|politics|(10000,[10,47,132,207,221,2...|[1.0,0.0,0.0,2.580050456943...|  0.0|       0.0|
|politics|(10000,[1,20,60,82,95,120,1...|             [1.0,0.0,0.0,0.0]|  0.0|       0.0|
|politics|

0.9387861118473364

## Calculate ACCURACY using Cross Validation on Logistic Regression
Spark MLlib provides for cross-validation for hyperparameter tuning. Cross-validation attempts to fit the underlying estimator with user-specified combinations of parameters, cross-evaluate the fitted models, and output the best one.

In [110]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
#from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3, 0.5]) # regularization parameter
             .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) #Number of iterations
#            .addGrid(idf.numFeatures, [10, 100, 1000]) # Number of features
             .build())
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator, \
                    numFolds=5)
pipelineCVLR = Pipeline(stages=[regexTokenizer, remover, hashingTF, idf, label_stringIdx,cv])
cvModel = pipelineCVLR.fit(trainingData)

predictions = cvModel.transform(testData)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)


0.9481706183750974

## Accuracy of UNKNOWN Data using CV with Logistic Regression
#### Keep in mind that the model has not seen the documents in the Unknown data set.

In [111]:
predictions = cvModel.transform(dfUN)
# Evaluate best model
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9796340896607091

## CLASSIFICATION USING "RANDOM FOREST"

In [112]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(labelCol="label", \
                            featuresCol="features", \
                            numTrees = 100, \
                            maxDepth = 4, \
                            maxBins = 32)
pipelineRF = Pipeline(stages=[regexTokenizer, remover, hashingTF, idf, label_stringIdx, rf])

# Train model with Training Data
rfModel = pipelineRF.fit(trainingData)
predictions = rfModel.transform(testData)
predictions.filter(predictions['prediction'] == 0) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|politics|(10000,[22,67,70,74,87,93,1...|[0.6262811921483312,0.13496...|  0.0|       0.0|
|politics|(10000,[1,15,24,43,70,72,87...|[0.625484320693428,0.157946...|  0.0|       0.0|
|politics|(10000,[1,15,78,87,138,150,...|[0.6142314841489602,0.16987...|  0.0|       0.0|
|politics|(10000,[44,47,126,132,137,1...|[0.6055520409583321,0.16298...|  0.0|       0.0|
|politics|(10000,[15,20,26,45,70,76,8...|[0.5981348959983048,0.14785...|  0.0|       0.0|
|politics|(10000,[15,87,126,132,145,1...|[0.579238978116515,0.167673...|  0.0|       0.0|
|politics|(10000,[12,18,21,45,169,213...|[0.5747873623922527,0.14981...|  0.0|       0.0|
|politics|(10000,[47,71,76,77,87,113,...|[0.5743978416685226,0.16819...|  0.0|       0.0|
|politics|

## Accuracy  of Test Data using "Random Forest"
#### Keep in mind that the model has not seen the documents in the test data set.

In [113]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.8061810402318066

## Accuracy  of UNKNOWN Data using "Random Forest"
#### Keep in mind that the model has not seen the documents in the Unknown data set.

In [114]:
predictions = rfModel.transform(dfUN)
predictions.filter(predictions['prediction'] == 0) \
    .select("Category","features","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

+--------+------------------------------+------------------------------+-----+----------+
|Category|                      features|                   probability|label|prediction|
+--------+------------------------------+------------------------------+-----+----------+
|politics|(10000,[1,20,60,82,95,120,1...|[0.5919684996671557,0.13488...|  0.0|       0.0|
|politics|(10000,[1,24,76,78,87,93,10...|[0.5458226461800758,0.13562...|  0.0|       0.0|
|politics|(10000,[10,47,132,207,221,2...|[0.541972058349105,0.171282...|  0.0|       0.0|
|politics|(10000,[1,21,24,29,69,77,92...|[0.524626965702046,0.167296...|  0.0|       0.0|
|politics|(10000,[21,103,115,126,130,...|[0.5021602611456732,0.20419...|  0.0|       0.0|
|politics|(10000,[15,103,157,161,253,...|[0.45473108742121454,0.2065...|  0.0|       0.0|
|politics|(10000,[78,165,207,216,273,...|[0.44781224279846,0.2173283...|  0.0|       0.0|
|politics|(10000,[1,87,115,126,147,16...|[0.42437671757453715,0.2296...|  0.0|       0.0|
|politics|

0.8011736273456751

## ASSESSMENT:
**(This analysis was intended to illustrate how to use the spark.ml machine learning package utilizing a machine learning pipeline on Data from a Public Resource like NY Times.)

#### The Program also shows the accuracy of the learning Model by Calculating the Accuracy of Unknown dataset (Data which has not been used in Training or Test Dataset)


#### Accuracy of Unknown Dataset on Various Models:
##### Logistic Regression: $0.918297808424174$
##### Naive Bayes : $0.9387861118473364$
##### Cross Validation Using Logistic Regression : $0.9796340896607091$
##### Random Forest : $0.8011736273456751$

#### Clearly the Cross Validation yeilds highest accuracy.
#### Random forest is not a good choice for high-dimensional sparse data.

### Conclusion: Logistic Regression Using Cross Validation is the best model in our analysis



### ------------------------------------------------------------------------------------------------------------###

## References:

##### https://www.youtube.com/watch?v=OednhGRp938&feature=youtu.be
##### https://yidatao.github.io/2016-03-23/Document-Classification-using-pyspark/
##### https://towardsdatascience.com/multi-class-text-classification-with-pyspark-7d78d022ed35
##### https://www.youtube.com/watch?v=YX1xxWVxM0w
##### https://ibm.ent.box.com/s/spcj7f3uz6qetq8442mnvw5j264wbilj